Patrick BROCKMANN - LSCE (Climate and Environment Sciences Laboratory)

<img align="left" width="40%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png"><br><br>
<hr>

Last update: 2016/06/17

## Investigate how to select images from their metadata (CRF_def)

In [1]:
# http://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/fabric/lmdz/MultiSimu/CMIP5to6/ATLAS/index_SH_YEAR_AR4.0ada_1982_1991.html
# http://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/fabric/lmdz/MultiSimu/CMIP5to6/ATLAS/index_GLOB_YEAR_NPv5.3_1982_1989.html

In [2]:
import os
import pandas as pd
import urllib2
from urlparse import urlparse
from bs4 import BeautifulSoup
from wand.image import Image

In [116]:
from IPython.core.display import display, HTML
#pd.set_option('display.max_colwidth', 300)    # default

### Harvest all png images from a set og html pages (atlas) 

Can be long since you have to download and open as many images there are in the different html pages 

In [87]:
df = pd.DataFrame(columns={'url', 'metadata', 'image'})

urlList = []
urlList.append('http://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/fabric/lmdz/MultiSimu/CMIP5to6/ATLAS/' + \
                   'index_SH_YEAR_AR4.0ada_1982_1991.html')
urlList.append('http://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/fabric/lmdz/MultiSimu/CMIP5to6/ATLAS/' + \
                   'index_GLOB_YEAR_NPv5.3_1982_1989.html')


for url in urlList:
    
    print 'Harvesting ' + url
    
    parsed_uri = urlparse(url)
    domain = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)

    html = urllib2.urlopen(url).read()

    soup=BeautifulSoup(html, "lxml")

    allImg = soup.findAll('img')

    n = 0
    for img in allImg:
        filename, file_extension = os.path.splitext(img['src'])
        if (file_extension == '.png'):
            imgURL = domain + img['src']
            response = urllib2.urlopen(imgURL)
            try:
                with Image(file=response) as img:
                    metadata = img.metadata['CRS_def']
                    imgHTML = '<a href="' + imgURL + '" target="_blank">' + \
                                '<img class="myImage" src="' + imgURL + '"/>' + \
                                '</a>'
                    df = df.append({'url': imgURL, 'metadata': metadata, 'image': imgHTML}, ignore_index=True)
                    n = n+1
            except:
                response.close()
                
    print '    ' + str(n) + ' png images found'
 
print 
print 'Total: ', len(df)

Harvesting http://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/fabric/lmdz/MultiSimu/CMIP5to6/ATLAS/index_SH_YEAR_AR4.0ada_1982_1991.html
    178 png images found
Harvesting http://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/fabric/lmdz/MultiSimu/CMIP5to6/ATLAS/index_GLOB_YEAR_NPv5.3_1982_1989.html
    178 png images found

Total:  356


### Remove duplicate urls

In [88]:
df = df.drop_duplicates()
print len(df)

252


In [100]:
from IPython.display import HTML
HTML("""
<style>
.myImage {
  width: 200px;
  float: left;
  border:1px solid black;
}
</style>
""")

### Examples of selection

In [112]:
# OR example
df_select = df[df['metadata'].str.contains('albt|tas')]

# AND examples
df_select = df[df['metadata'].str.contains('albt') & \
               df['metadata'].str.contains('curves')]

df_select = df[df['metadata'].str.contains('albt') & \
               df['metadata'].str.contains('SH')]

df_select = df[df['metadata'].str.contains('albt|tas')]

### Display images in the dataframe

In [117]:
display(HTML(df_select[['image']].to_html(escape=False, index=False)))

image
""
""
""
""
""
""
""
""
""
""


### Display images as block

In [118]:
a= df_select[['image']].values
b = [item for sublist in a for item in sublist]

display(HTML('<div style="display: block">' + ''.join(b) + '</div>'))